## To Do:
- Convert the non-gui implementation in main.py to use a gui
- Use [tkinter documentation](https://docs.python.org/3/library/tk.html)
- Implement the following features:
    - **Timeline Scrubbing**
        - View edits as highlighted sections on the timeline
        - Multiple edits are shown in horizontal strips, stacked on top of each other, with identifying colors, text, and relevant info
        - Drag the edges of a strip to change the duration
    - **Edit Durations**
        - Input the number of frames a particular edit should be duplicated across.
    - **Edit Transitions**
        - Global edits can be set to transition over a number of frames
        - Transition interpolations (Linear, Cosine, Cubic)
    - **Copy/Paste Selection**
        - Click the tool icon
        - Drag or Click two points, A & B, to select a rectangle.
            - Hold CTRL+Drag to get a perfect square
        - Drag the corners to edit the selection
        - Press Enter to Select
    - **Paint Tools**
        - Onion Skin Option
        - Brush Size, Opacity, 
    - **Select Ellipse**
        - Hold CTRL+Drag to get a perfect circle
    - **Select Lasso**
        - Drag the mouse around to select something
    - **Color Select**
        - Threshold
    - **HSL**
        - Hue, Saturation, Lightness
    - **BC**
        - Brightness, Contrast
    - **Import GIF with URL**
    - **Export GIF**
- Stretch Goals
    - 3D Animated Templates
        - Cube Spin
- Come up with more ideas for features; refer to a dumb non-moving image editor like MS Paint, or [Pixlr E](https://pixlr.com/e/) for inspiration.